<a href="https://colab.research.google.com/github/shreyasguha/Shreyas_AI/blob/main/numpy_digit_classifier/MNIST_digitclassifier_1layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple MNIST NN using numpy

Here I implement a simple neural network with 1 input layer, 1 hidden layer, and 1 output layer for classifying the MNIST handwritten digit recognition database. The hidden layer consists of 10 neurons, just like the output layer

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

**Importing the data**

In [6]:
pd_data = pd.read_csv('/content/drive/MyDrive/AI/Public Dataset/MNIST digit database/mnist_train.csv')
data = np.array(pd_data)
np.random.shuffle(data)

**Dividing the data into training and dev**

In [7]:
m, n = data.shape

data_dev = data[0:1000].T
# Y_dev has 1 row, and columns from 1 to 1000 each column containing the actual label of that datapoint
Y_dev = data_dev[0]
# X_dev has 785 rows, each corresponding to different pixel values, and 1000 columns, each for a different datapoint
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
# Y_train has 1 row, and columns from 1000 to 60000 each column containing the actual label of that datapoint
Y_train = data_train[0]
# X_train has 785 rows, each corresponding to different pixel values, and 59000 columns, each for a different datapoint
X_train = data_train[1:n]
X_train = X_train / 255.
#_,m_train = X_train.shape

In [8]:
X_train.shape

(784, 59000)

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = softmax(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x 1
- $Z^{[1]} \sim A^{[1]}$: 10 x 1
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x 1
- $W^{[2]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1



**Define usefult functions such as parameter initialization, ReLU, softmax, forward propogation, derivative of ReLU, one hot encoding of Y, backward propogation and parameter update**

In [12]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

**Define some more important functions such as get predictions (to convert output of model A2 to something that can be compared with the label data), accuracy calculator, make_predictions and test prediction (which can pass a specific input through the model and show us the output), and 2 functions for gradient descent, namely gradient_descent_start and gradient_descent_modify which can be used to start the process of training the model and further finetune the model respectively**

In [10]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent_start(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

def gradient_descent_modify(W1, b1, W2, b2, X, Y, alpha, iterations):
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
W1, b1, W2, b2 = gradient_descent_start(X_train, Y_train, 0.10, 500)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

- On 500 timesteps we get 83.1% accuracy on our training data and 82.5% accuracy on our test data
- 500 more timesteps, train:87.4% test:87.3%
- 340 more timesteps to reach train:90% test:89.7%
- 1500 more timesteps, train:92.2% test:91%
- 2000 more timesteps, train:93.3% test:92.3%
- 4000 more timesteps, train:93.9% test:92.4%

Therefore we conclude that the accuracy of this single layer neural network peaks at about 92.4% on test data
